# Hands sync

This is the fourth part of my project, in this part we will try to make a classifier for hand movement, to see if it was alone, syncronized or not

In [1]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import glob
import random
%matplotlib inline
from PIL import Image

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [45]:
#function to add tags to the rows. 0 will be alone, 1 is spontanious and 2 is syncronized
def add_tag(dataframe, path):
    if "Spontan" in path:
        tags = np.ones(dataframe.shape[0],dtype=int)
        dataframe['Label'] = tags
    elif "alone" in path:
        tags = np.zeros(dataframe.shape[0],dtype=int)
        dataframe['Label'] = tags
    else:
        tags = np.full(dataframe.shape[0], 2,dtype = int)
        dataframe['Label'] = tags


In [47]:
#function to merge all rows in a file that have the same second but diffrent hands
def merge_right_left(path):
    df = pd.read_csv(path + ".csv")
    right = df.loc[df[" Hand Type"] == "right"]
    left = df.loc[df[" Hand Type"] == "left"]
    for column in right.columns: #lets rename the columns so we know which is right nad which is left
        right = right.rename(columns = {column: column + "_R"})
    for column in left.columns:
        left = left.rename(columns = {column: column + "_L"})
    merged = right.merge(left, left_on="Time_R", right_on = "Time_L")
    add_tag(merged, path)
    return merged

First, lets get one of the files and see what it looks like

In [24]:
df = pd.read_csv("D:\ML_SEM2\extraFiles\Training\All in one\Evyatar636771052727603804Spontan.csv")

In [25]:
df.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,335.9984,128556,right,2,128.63570,224.0294,-12.29588,-18.26243,-140.54450,-117.57790,...,-1.588251,177.5125,173.5658,-14.569630,276.8753,-33.68264,91.61740,0.0,1.250576,0.0
1,335.9984,128556,left,2,-93.24845,143.9423,-37.81597,-109.09820,240.38980,-67.62959,...,0.873458,-147.0455,109.9380,-5.326688,-329.8577,-34.41571,93.88171,0.0,0.014736,0.0
2,336.0150,128558,right,2,129.15550,221.9923,-15.00181,47.36464,-64.25022,-168.85850,...,-1.528183,177.7894,171.1952,-14.677850,276.0645,-33.15913,97.88171,0.0,1.069930,0.0
3,336.0150,128558,left,2,-94.86554,148.5542,-39.00158,-80.99012,278.60220,-66.79356,...,0.879703,-148.2022,113.0909,-7.375025,-326.9279,-39.97873,85.98000,0.0,0.000000,0.0
4,336.0315,128560,right,2,129.87070,220.7944,-18.38681,35.05422,-84.74242,-205.72030,...,-1.461710,178.3730,169.9159,-15.246240,276.1767,-30.72506,104.13920,0.0,0.916969,0.0


In [43]:
df.isnull().sum()

Time               0
 Frame ID          0
 Hand Type         0
 # hands           0
 Position X        0
 Position Y        0
 Position Z        0
 Velocity X        0
 Velocity Y        0
 Velocity Z        0
 Pitch             0
 Roll              0
 Yaw               0
 Wrist Pos X       0
 Wrist Pos Y       0
 Wrist Pos Z       0
 Elbow pos X       0
 Elbow Pos Y       0
 Elbow Pos Z       0
 Grab Strenth      0
 Grab Angle        0
 Pinch Strength    0
dtype: int64

In [28]:
df.columns

Index(['Time', ' Frame ID', ' Hand Type', ' # hands', ' Position X',
       ' Position Y', ' Position Z', ' Velocity X', ' Velocity Y',
       ' Velocity Z', ' Pitch', ' Roll', ' Yaw', ' Wrist Pos X',
       ' Wrist Pos Y', ' Wrist Pos Z', ' Elbow pos X', ' Elbow Pos Y',
       ' Elbow Pos Z', ' Grab Strenth', ' Grab Angle', ' Pinch Strength'],
      dtype='object')

In [48]:
merged = merge_right_left("D:\ML_SEM2\extraFiles\Training\All in one\Evyatar636771052727603804Spontan")

In [49]:
merged.head()

,Time_R,Frame ID_R,Hand Type_R,# hands_R,Position X_R,Position Y_R,Position Z_R,Velocity X_R,Velocity Y_R,Velocity Z_R,...,Wrist Pos X_L,Wrist Pos Y_L,Wrist Pos Z_L,Elbow pos X_L,Elbow Pos Y_L,Elbow Pos Z_L,Grab Strenth_L,Grab Angle_L,Pinch Strength_L,Label
0,335.9984,128556,right,2,128.6357,224.0294,-12.29588,-18.26243,-140.54450,-117.5779,...,-147.0455,109.9380,-5.326688,-329.8577,-34.41571,93.88171,0.0,0.014736,0.0,1
1,336.0150,128558,right,2,129.1555,221.9923,-15.00181,47.36464,-64.25022,-168.8585,...,-148.2022,113.0909,-7.375025,-326.9279,-39.97873,85.98000,0.0,0.000000,0.0,1
2,336.0315,128560,right,2,129.8707,220.7944,-18.38681,35.05422,-84.74242,-205.7203,...,-148.3343,115.9295,-9.437292,-322.0559,-44.42383,81.02193,0.0,0.000000,0.0,1
3,336.0481,128562,right,2,131.0721,219.2134,-22.05511,36.51635,-153.35370,-212.8259,...,-148.3562,118.3093,-10.919770,-314.2976,-49.04110,81.41824,0.0,0.000000,0.0,1
4,336.0651,128564,right,2,132.4629,216.8003,-25.45043,98.73232,-65.53646,-218.7201,...,-149.2292,121.7602,-12.780660,-299.9106,-56.55378,85.03669,0.0,0.000000,0.0,1
